In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

In [4]:
%cd "Documents/GitHub/pmx_gasoline_theft_prediction/"

/Users/jorgeroa/Documents/GitHub/pmx_gasoline_theft_prediction


In [63]:
df_homics_19 = pd.read_parquet("data/df_homics_19.parquet")
df_PMX_19_thefts_pp = pd.read_parquet('data/df_PMX_19_thefts_pp_final.parquet')
df_PMX_gas_place_price = pd.read_parquet('data/df_places_prices_final.parquet')
df_social_2019 = pd.read_excel('data/df_social_19_final_JR.xlsx')
df_SEDENA_19 = pd.read_parquet('data/df_SEDENA_19.parquet')

df_pipeline_total = pd.read_excel('data-raw/df_pipelines.xlsx')


df_cars_2019 = pd.read_parquet('data/df_cars_2019.parquet')
df_rural_w_group = pd.read_parquet('data/df_rural_w_group.parquet')
df_highway_access = pd.read_parquet('data/df_highway_access.parquet')
df_density_km = pd.read_parquet('data/df_density_km.parquet')




df_cvegeo = pd.read_excel("data-raw/codigo municipio inegi.xlsx")
df_cvegeo = df_cvegeo.rename(columns = {"codigo inegi":"CVEGEO"})

df_cvegeo['CVEGEO'] = df_cvegeo['CVEGEO'].astype(str).str.zfill(7).str.replace('.',"")
df_cvegeo['CVEGEO'] = df_cvegeo['CVEGEO'].str[:-1]
df_cvegeo_f = df_cvegeo[["CVEGEO", "NOM_ENT", "NOM_MUN"]]

df_pipeline_total['CVEGEO']=df_pipeline_total['CVEGEO'].astype(object)
df_pipeline_total['CVEGEO'] = df_pipeline_total['CVEGEO'].astype(str).str.zfill(5)



df_social_2019['CVEGEO']=df_social_2019['CVEGEO'].astype(object)
df_social_2019['CVEGEO'] = df_social_2019['CVEGEO'].astype(str).str.zfill(5)

df_PMX_19_thefts_pp

/var/folders/q3/ztffc4r10tv5h4pftjr2qw0r0000gn/T/ipykernel_25587/557121668.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_cvegeo['CVEGEO'] = df_cvegeo['CVEGEO'].astype(str).str.zfill(7).str.replace('.',"")


,CVEGEO,type_inegi,soil_inegi,pp_inegi,cap_5ypp,inv_5ypp,long_5ypp,pp_5ypp,pp_instal,pp_integ_sistran_pp,...,pp_SISTRANGAS_pipeline,pp_st_term,type_st_term,cap_st_term,cap_st_term_1,pipeline,sum_intersects,year,Num_thefts,_merge
0,09012,UN CONDUCTO,SUBTERRANEO,2,None,None,NaN,0,0,0,...,0,1,Poliducto,23000.0,22000.0,1,4,2019.0,1.0,both
1,09013,UN CONDUCTO,SUBTERRANEO,3,None,None,NaN,0,0,0,...,0,1,Poliducto,23000.0,22000.0,1,5,NaN,NaN,left_only
2,09008,UN CONDUCTO,SUBTERRANEO,1,None,None,NaN,0,0,0,...,0,0,None,NaN,NaN,1,1,NaN,NaN,left_only
3,09002,UN CONDUCTO,SUBTERRANEO,3,None,None,NaN,0,0,0,...,0,5,Turbosino-ducto,428000.0,412000.0,1,13,2019.0,8.0,both
4,09014,None,None,0,None,None,NaN,0,0,0,...,0,0,None,NaN,NaN,0,0,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2451,23007,None,None,0,None,None,NaN,1,0,0,...,0,0,None,NaN,NaN,1,3,NaN,NaN,left_only
2452,23003,None,None,0,None,None,NaN,0,0,0,...,0,0,None,NaN,NaN,0,0,NaN,NaN,left_only
2453,15108,UN CONDUCTO,SUBTERRANEO,2,None,None,NaN,0,0,1,...,1,0,None,NaN,NaN,1,4,NaN,NaN,left_only
2454,15059,UN CONDUCTO,SUBTERRANEO,2,None,None,NaN,0,0,1,...,1,0,None,NaN,NaN,1,4,2019.0,41.0,both


In [41]:
df_cvegeo_f

,CVEGEO,NOM_ENT,NOM_MUN
0,01001,Aguascalientes,Aguascalientes
1,01002,Aguascalientes,Asientos
2,01003,Aguascalientes,Calvillo
3,01004,Aguascalientes,Cosío
4,01005,Aguascalientes,Jesús María
...,...,...,...
2451,32054,Zacatecas,Villa Hidalgo
2452,32055,Zacatecas,Villanueva
2453,32056,Zacatecas,Zacatecas
2454,32057,Zacatecas,Trancoso


In [42]:
df_homics_19.dtypes

CVEGEO      object
homic_19     int64
dtype: object

In [43]:
df_PMX_19_thefts_pp.dtypes

CVEGEO                        object
type_inegi                    object
soil_inegi                    object
pp_inegi                       int64
cap_5ypp                      object
inv_5ypp                      object
long_5ypp                    float64
pp_5ypp                        int64
pp_instal                      int64
pp_integ_sistran_pp            int64
long_integ_sistran_pp        float64
type_integ_sistran_pp         object
pp_mix_pp                      int64
long_mix_pp                  float64
type_tube_mix_pp              object
type_gas_mix_pp               object
pp_n_int_sist                  int64
long_n_int_sist              float64
type_n_int_sist               object
cap_n_int_sist               float64
inv_n_int_sist               float64
pp_oil                         int64
long_oil                     float64
type_oil                      object
cap_oil                      float64
cap_oil_2                    float64
cap_pp_tran_sys               object
i

In [62]:
df_pipeline_total

,CVEGEO,pipe
0,1001,1
1,1002,1
2,1004,1
3,1005,1
4,1006,1
...,...,...
834,32050,1
835,32051,1
836,32052,1
837,32056,1


In [60]:
df_PMX_19_thefts_pp_final = pd.merge(df_PMX_19_thefts_pp,
                             df_pipeline_total,
                             on='CVEGEO',
                             how='outer',
                             indicator=True)

ValueError: Cannot use name of an existing column for indicator column

In [64]:


dataframes = [df_cvegeo_f,
              df_pipeline_total,
              df_PMX_19_thefts_pp,
              df_PMX_gas_place_price,
              df_homics_19,
              df_SEDENA_19,
              df_social_2019,
              df_cars_2019,
              df_rural_w_group,
              df_highway_access,
              df_density_km]




df_PMX_thefts_19 = reduce(lambda  left,right: pd.merge(left,right,on=['CVEGEO'],
                                            how = 'outer'), dataframes)#.fillna(0)

In [65]:
df_PMX_thefts_19

,CVEGEO,NOM_ENT,NOM_MUN,pipe,type_inegi,soil_inegi,pp_inegi,cap_5ypp,inv_5ypp,long_5ypp,...,pob_2020,pob_highway,porc_highway,CARR-KM,LINCARR,RED-KM,LINRED,AREAKM,DENSCARR,DENSRED
0,01001,Aguascalientes,Aguascalientes,1.0,UN CONDUCTO,SUBTERRANEO,8.0,"1,189",473,600.0,...,948990.0,1023.0,0.1,438.190142,1053.0,765.173487,5500.0,1178.10,0.371946,0.649498
1,01002,Aguascalientes,Asientos,1.0,None,None,0.0,886,294,305.0,...,51536.0,175.0,0.3,173.662987,284.0,158.986649,615.0,549.01,0.316320,0.289588
2,01003,Aguascalientes,Calvillo,NaN,None,None,0.0,None,None,NaN,...,58250.0,601.0,1.0,114.439835,207.0,193.630319,598.0,932.74,0.122692,0.207593
3,01004,Aguascalientes,Cosío,1.0,None,None,0.0,"1,189",473,600.0,...,17000.0,3.0,0.0,69.959236,108.0,28.910152,255.0,129.74,0.539226,0.222831
4,01005,Aguascalientes,Jesús María,1.0,None,None,0.0,"1,189",473,600.0,...,129929.0,59.0,0.0,151.549502,438.0,177.436744,1292.0,504.99,0.300104,0.351367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,07123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10783.0,833.0,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2485,07125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11650.0,9840.0,84.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2486,17034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11347.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2487,17035,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27805.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
df_PMX_thefts_19 = df_PMX_thefts_19[(df_PMX_thefts_19['pipe']==1) & (df_PMX_thefts_19['CVEGEO'].str.len()>=5)]

In [67]:
df_PMX_thefts_19["homic_rate"] = df_PMX_thefts_19["homic_19"]/df_PMX_thefts_19["POB_TOT"]*100000
df_PMX_thefts_19["vehic_rate"] = df_PMX_thefts_19["vehicles"]/df_PMX_thefts_19["POB_TOT"]*100000

/var/folders/q3/ztffc4r10tv5h4pftjr2qw0r0000gn/T/ipykernel_25587/1261863900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PMX_thefts_19["homic_rate"] = df_PMX_thefts_19["homic_19"]/df_PMX_thefts_19["POB_TOT"]*100000
/var/folders/q3/ztffc4r10tv5h4pftjr2qw0r0000gn/T/ipykernel_25587/1261863900.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PMX_thefts_19["vehic_rate"] = df_PMX_thefts_19["vehicles"]/df_PMX_thefts_19["POB_TOT"]*100000


In [68]:

df_PMX_thefts_19['confront']=df_PMX_thefts_19['confront'].fillna(0)

df_PMX_thefts_19_f = df_PMX_thefts_19.drop(['_merge_x', '_merge_y', "pipeline"], axis=1)


df_PMX_thefts_19_f['pipeline'] = np.select([(df_PMX_thefts_19_f['Num_thefts'] >= 0)], [1])



df_PMX_thefts_19_f['pipeline'].sum()



/var/folders/q3/ztffc4r10tv5h4pftjr2qw0r0000gn/T/ipykernel_25587/4090929529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_PMX_thefts_19['confront']=df_PMX_thefts_19['confront'].fillna(0)


237

In [69]:
df_PMX_thefts_19_f

,CVEGEO,NOM_ENT,NOM_MUN,pipe,type_inegi,soil_inegi,pp_inegi,cap_5ypp,inv_5ypp,long_5ypp,...,CARR-KM,LINCARR,RED-KM,LINRED,AREAKM,DENSCARR,DENSRED,homic_rate,vehic_rate,pipeline
0,01001,Aguascalientes,Aguascalientes,1.0,UN CONDUCTO,SUBTERRANEO,8.0,"1,189",473,600.0,...,438.190142,1053.0,765.173487,5500.0,1178.10,0.371946,0.649498,7.279150,50770.251826,1
1,01002,Aguascalientes,Asientos,1.0,None,None,0.0,886,294,305.0,...,173.662987,284.0,158.986649,615.0,549.01,0.316320,0.289588,1.981022,28409.833792,0
3,01004,Aguascalientes,Cosío,1.0,None,None,0.0,"1,189",473,600.0,...,69.959236,108.0,28.910152,255.0,129.74,0.539226,0.222831,NaN,30200.538761,0
4,01005,Aguascalientes,Jesús María,1.0,None,None,0.0,"1,189",473,600.0,...,151.549502,438.0,177.436744,1292.0,504.99,0.300104,0.351367,7.814576,41965.053217,0
5,01006,Aguascalientes,Pabellón de Arteaga,1.0,UN CONDUCTO,SUBTERRANEO,1.0,"1,189",473,600.0,...,85.510623,239.0,92.675448,682.0,197.76,0.432396,0.468626,2.109304,34331.034192,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447,32050,Zacatecas,Vetagrande,1.0,None,None,0.0,"1,189",473,600.0,...,66.247353,120.0,14.317158,192.0,159.96,0.414149,0.089505,68.775791,28424.051877,0
2448,32051,Zacatecas,Villa de Cos,1.0,None,None,0.0,None,None,NaN,...,460.761886,277.0,297.153257,629.0,6669.43,0.069086,0.044555,37.412225,19886.036606,0
2449,32052,Zacatecas,Villa García,1.0,UN CONDUCTO,SUBTERRANEO,1.0,886,294,305.0,...,69.377975,96.0,62.400043,260.0,341.74,0.203014,0.182595,5.163689,28322.833832,0
2453,32056,Zacatecas,Zacatecas,1.0,UN CONDUCTO,SUBTERRANEO,1.0,None,None,NaN,...,125.274509,299.0,132.960053,1698.0,441.50,0.283747,0.301155,63.797353,49771.336857,0


In [70]:
df_PMX_thefts_19_f.to_excel('data/df_PMX_thefts_19_pp_final.xlsx', index = False)
df_PMX_thefts_19_f.to_parquet('data/df_PMX_thefts_19_pp_final.parquet')